In [2]:
import pandas as pd
import numpy as np
import joblib
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from xgboost import XGBRegressor
from datetime import datetime
from sklearn.metrics import mean_absolute_error

df = pd.read_csv("Veri/tum_veri.csv")

df["time"] = pd.to_datetime(df["time"])
df["gun"] = df["time"].dt.day
df["ay"] = df["time"].dt.month
df["hafta_gunu"] = df["time"].dt.dayofweek

df = df.dropna()

encoder = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
il_encoded = encoder.fit_transform(df[["il"]])
il_df = pd.DataFrame(il_encoded, columns=encoder.get_feature_names_out(["il"]))
il_df.reset_index(drop=True, inplace=True)

X = pd.concat([df[["gun", "ay", "hafta_gunu"]].reset_index(drop=True), il_df], axis=1)
y = df[["temperature_2m_max", "temperature_2m_min", "precipitation_sum"]]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = MultiOutputRegressor(
    XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=6, random_state=42)
)
model.fit(X_train, y_train)

joblib.dump(model, "hava_durumu_model.pkl")
joblib.dump(encoder, "encoder.pkl")

print("Model eğitildi ve kaydedildi.")

# --- test için
def kullanici_tahmini_al():
    il = input("İl adı girin (örn: Ankara): ").strip()
    tarih = input("Tarih girin (GG-AA-YYYY): ").strip()

    try:
        tarih = datetime.strptime(tarih, "%d-%m-%Y")
        gun = tarih.day
        ay = tarih.month
        hafta_gunu = tarih.weekday()

        encoder = joblib.load("encoder.pkl")
        il_encoded = encoder.transform(pd.DataFrame([[il]], columns=["il"]))
        il_df = pd.DataFrame(il_encoded, columns=encoder.get_feature_names_out(["il"]))

        x_input = pd.concat([pd.DataFrame([[gun, ay, hafta_gunu]], columns=["gun", "ay", "hafta_gunu"]), il_df], axis=1)

        model = joblib.load("hava_durumu_model.pkl")
        tahmin = model.predict(x_input)

        print(f"\n📍 {il} için {tarih.strftime('%d %B %Y')} tahmini:")
        print(f"🔺 Maksimum Sıcaklık: {tahmin[0][0]:.1f} °C")
        print(f"🔻 Minimum Sıcaklık: {tahmin[0][1]:.1f} °C")
        print(f"🌧️  Yağış Miktarı:     {tahmin[0][2]:.1f} mm\n")

    except Exception as e:
        print(f"Hata: {e}")

y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(f" MAE (ortalama): {mae:.2f} °C")

kullanici_tahmini_al()


Model eğitildi ve kaydedildi.
 MAE (ortalama): 2.19 °C


İl adı girin (örn: Ankara):  Ankara
Tarih girin (GG-AA-YYYY):  09-04-2024



📍 Ankara için 09 April 2024 tahmini:
🔺 Maksimum Sıcaklık: 14.3 °C
🔻 Minimum Sıcaklık: 5.5 °C
🌧️  Yağış Miktarı:     3.7 mm

